In [28]:
#USE GOOGLE DRIVE FOR FOLES TO 
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [29]:
#SPECIFIC VERSIONS OF TENSORFLOW AND KERAS ARE REQUIRED TO RUN WITH DARKNET AND MY MODEL
#TRIED TO USE THE KERAS MODEL IN THE INSTRUCTIONS BUT IT WAS BUILT ON THE PREVIOUS TENSORFLOW
#WITH MY MODEL BEING BUILT ON TENSORFLOW 2.0 THERE WAS HUGE ISSUES WITH TRYING TO USE THE MODEL

!pip install tensorflow==2.4.0
!pip install keras==2.1.5 

import tensorflow as tf

import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [36]:
#HERE WE CREATE THE TRAINING AND VALIDATION DATA FROM THE FOLDERS OF IMAGES THAT I HAVE FOR MY PROJECT
#I HAVE A TOTAL OF 800 OF EACH TYPE OF IMAGE

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/gdrive/MyDrive/dataset/dataset2/",
    labels="inferred",
    label_mode="categorical",  # categorical, binary
    class_names=['sedan', 'suv'],
    color_mode="rgb",
    batch_size=32,
    image_size=(416, 416),  # reshape if not in this size
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="training",
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/gdrive/MyDrive/dataset/dataset2/",
    labels="inferred",
    label_mode="categorical",  # categorical, binary
    class_names=['sedan', 'suv'],
    color_mode="rgb",
    batch_size=32,
    image_size=(416, 416),  # reshape if not in this size
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="validation",
)

Found 1600 files belonging to 2 classes.
Using 1280 files for training.
Found 1600 files belonging to 2 classes.
Using 320 files for validation.


In [37]:
batch_size = 32

#HERE IS WHERE E DO SOME DATA AUGMENTATION
#WE DO A RANDOM HORIZONTAL FLIPS AND ROTATION ON SOME OF THE DATA 
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

In [38]:
# MOBILENET MODE TRAINED ON IMAGENT DATASET
base_model = keras.applications.MobileNet(
    weights="imagenet",  
    input_shape=(416, 416, 3),
    include_top=False,
)  

print(train_ds)

# KEEP THE BASE MODEL FOR LATER DONT TRAIN STRAIGHT AWAY
base_model.trainable = False

# HERE WE CREATE THE NEW MODEL ON TOP OF THE BASE MODEL 
inputs = keras.Input(shape=(416, 416, 3))
# HERE WE APPLY THE DATA AUGMENTATION AT RANDOM
x = data_augmentation(inputs) 


#HERE WE DO SOME NORMALIZATION OF THE INPUT IMAGE DATA TO (-1,1)
norm_layer = keras.layers.experimental.preprocessing.Normalization()
mean = np.array([127.5] * 3)
var = mean ** 2
x = norm_layer(x)
norm_layer.set_weights([mean, var])

# WE WILL KEEP THE BASE MODEL FROM TRAINING SO WE CAN DO SOME FINE TUNING LATER
x = base_model(x, training=False)
# USE AVERAGE POOLING LAYER TO 
# REDUCE SIZE OF REPRESENTATION AND SPEED UP COMPUTATION
x = keras.layers.GlobalAveragePooling2D()(x)
#IMPLEMENT RANDOM DROPOUT OF SOME OF THE NN NODES 
#MAKES THE NN NOT RELY ON ANY ONE FEATURE AS IT CAN GO AWAY BASICALLY 
x = keras.layers.Dropout(0.2)(x)
# ADD A SIMPLE DENSE LAYER WITH TWO OUTPUTS 
outputs = keras.layers.Dense(2)(x)
# NOW WE HAVE THE MODEL
model = keras.Model(inputs, outputs)

model.summary()

<BatchDataset shapes: ((None, 416, 416, 3), (None, 2)), types: (tf.float32, tf.float32)>
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 416, 416, 3)]     0         
_________________________________________________________________
sequential_3 (Sequential)    (None, 416, 416, 3)       0         
_________________________________________________________________
normalization_2 (Normalizati (None, 416, 416, 3)       7         
_________________________________________________________________
mobilenet_1.00_224 (Function (None, 13, 13, 1024)      3228864   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1024)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
____________________________________

In [39]:
# HERE WE USE ADAM INSTEAD OF SGD
# BinaryCrossentropy AS OUR LOSS FUNCTION
# AND BINARY ACCURACY AS OUT ACCURACY METRIC

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

model.summary()

print(train_ds)

##WE DO SOME TRAINING 
epochs = 20
model.fit(train_ds, epochs=epochs, validation_data=val_ds)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 416, 416, 3)]     0         
_________________________________________________________________
sequential_3 (Sequential)    (None, 416, 416, 3)       0         
_________________________________________________________________
normalization_2 (Normalizati (None, 416, 416, 3)       7         
_________________________________________________________________
mobilenet_1.00_224 (Function (None, 13, 13, 1024)      3228864   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1024)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2050

In [40]:
#NOW WE TRAIN THE BASE MODEL AS A MEANS OF DOING SOME FINE TUNING TRAINING
#SO ESENTIALLY WE TRAIN USING OUR MODEL ON MY TRAINING DATA
#THEN WE FINE TUNE THIS USING THE BASE MODEL (MOBILENET)
#ALSO USE A VERY LOW LEARNING RATE 
base_model.trainable = True
model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(1e-5), 
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 10
model.fit(train_ds, epochs=epochs, validation_data=val_ds)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 416, 416, 3)]     0         
_________________________________________________________________
sequential_3 (Sequential)    (None, 416, 416, 3)       0         
_________________________________________________________________
normalization_2 (Normalizati (None, 416, 416, 3)       7         
_________________________________________________________________
mobilenet_1.00_224 (Function (None, 13, 13, 1024)      3228864   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1024)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2050

In [12]:
#SAVE THE MODEL SO I DON'T HAVE TO KEEP RUNNING IT 
model.save('saved_model/my_model_new_softmax1')

INFO:tensorflow:Assets written to: saved_model/my_model_new_softmax1/assets


In [41]:
#LOAD THE OLD SAVED MODEL 
#THIS IS WHEN MY COLAB TIMES OUT 
new_model = tf.keras.models.load_model('/content/gdrive/MyDrive/my_model_new_softmax')

In [13]:
#CLONE THE DARKNET LIBRARY
#LOAD IN THE WEIGHTS FOR TINY YOLO 

!git clone https://github.com/AlexeyAB/darknet.git
!wget https://pjreddie.com/media/files/yolov3-tiny.weights


Cloning into 'darknet'...
remote: Enumerating objects: 14748, done.
remote: Total 14748 (delta 0), reused 0 (delta 0), pack-reused 14748
Receiving objects: 100% (14748/14748), 13.29 MiB | 22.34 MiB/s, done.
Resolving deltas: 100% (10031/10031), done.
--2021-03-30 11:09:14--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35434956 (34M) [application/octet-stream]
Saving to: ‘yolov3-tiny.weights’

yolov3-tiny.weights 100%[===================>]  33.79M  15.2MB/s    in 2.2s    

2021-03-30 11:09:16 (15.2 MB/s) - ‘yolov3-tiny.weights’ saved [35434956/35434956]



In [42]:
#I HAD TO MAKE NUMEROUR CHANGES TO THE DARKNET PROJECT TO 
#1 ALLOW IT TO BE IMPORTED AS A LIBRARY
#2 GET THE LIBRRY TO WORK EITH MY IMAGES 
#ALSO I HAD TO CHANGE TINY-YOLOT.CFG FILE TO CREATE CORRECT BOUDING BOXES AS INITIALLY
#THEY ONLY COVERED A TINY PART OF MY VEHICLES

#THEREFOR HERE I NEED TO UPLOAD
#1 A NEW DARKNET.PY FILE
#2 A NEW MAKEFILE
#3 A NEW TINY-YOLO.CFG FILE ALL BEFORE THE NEXT MAKE STEP

%cd /content/darknet
#NEED TO CHANGE FILES BEFORE EXECUTING NEXT CELL

/content/darknet


In [16]:
#NEED TO MAKE CHANGES DON'T MAKE
#MAKE THE NEW DARKNET PROJECT 
######################################################################################################
##PLEASE LOOK AT INSTRUCTIONS FILE IN ZIP FILE BEFORE MAKING
######################################################################################################
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/

In [43]:
#IMPORT THE PROJECTS DARKNET.PY FILE
import darknet as dn


In [44]:
import importlib
importlib.reload(dn)

<module 'darknet' from '/content/darknet/darknet.py'>

In [45]:
#HERE WE CREATE A SOFTMAX LAYER FOR PREDICTION
probability_model = tf.keras.Sequential([new_model, tf.keras.layers.Softmax()])

In [67]:
import cv2 as cv
from PIL import Image
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import timeit
import sys 
start = 0
#THIS IS THE VERSION WITHOUT THE MUTITHREADING OPTIMIZATION 

#THIS IS THE METHOD THAT IS CALLED WHEN WE ARE LOOKING TO PARSE THE VIDEO

throughput_times = []
yolo_times = []
full_frame_times = []
classifier_times = []
q1_times = []
q2_times = []
crop_times = []

def getFrame(sec,cval, net, class_names, colors, lis, start, old_time):

    #THIS IS THE VIDEO READER
    #HERE WE USE THE SEC VALUE WHICH INCREMENTS THE AMOUNT WE NEED TO GET 900
    #FRAMES FROM THE GIVEN VIDEO 
    framestart = timeit.default_timer()
    q1start = timeit.default_timer()

    vidcap.set(cv.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    total_vehicles = 0
    sedan = 0
    suv = 0

    #ENTER IF THERE IS A FRAME AND WE HAVEN'T READ MORE THAN 900 FRAMES 
    if hasFrames and cval <= 900:

      #WRITE THE FRAME TO DISK 
      cv.imwrite("/content/gdrive/MyDrive/images/image"+str(count)+".jpg", image)     

      #GET THE PATH OF THE FRAME AS MY YOLO TAKES THE PATH AND LOOKS UP THE FILE
      #THUS WE HAVE TO WRITE TO DISK AND RE READ IN YOLO
      path = "/content/gdrive/MyDrive/images/image"+str(count)+".jpg"

      q1stop = timeit.default_timer()
      q1_times.append(q1stop - q1start)
      q2start = timeit.default_timer()


      #HERE Q2 STARTS SO 

      #GET THE IMAGE AND THE BOUNDING BOX LOCATIONS FROM YOLO
      x, bb = YOLO(path, cval, net, class_names, colors)
      image_list = []
      val = 0
      total_vehicles = 0
      sedan = 0
      suv = 0

      #LOOP THROUGH THE BOUNDING BOX LOCATIONS 
      for b in bb:
        if (b[0] == "car"):
          #GET THE LOCATIONS AND CROP OUT THE PROPOSED REGIONS FOR THE CAR
          left, top, right, bottom = dn.bbox2points(b[2])
          image_list.append(cropRegions(image, left,right,top,bottom, val))
          val = val + 1
      i = 0

      #LOOP THROUGH THE CROPPED IMAGES 
      for image in image_list:
        total_vehicles = total_vehicles + 1
        #SEND EACH IMAGE TO BE CLASSIFIED
        sedanres, suvres = classify(image, i)
        #INCREMEMNT THE RESULTS BASED ON WHAT IS CLASSIFIED
        sedan = sedan + sedanres
        suv = suv + suvres
        i = i + 1

      #CREATE A TUPLE WITH THE RESULTS FOR THE FRAME 
      x = (sedan, suv, total_vehicles)
      #APPEND THE RESULTS TO A LIST 
      lis.append(x)

      time = timeit.default_timer()


      #IF WE HAVE DONE 900 FRAMES THEN WE STOP THE TIMER AND RETURN THE THROUGHPUT
      if(cval != 1):
        throughput_times.append((time - old_time))
      

      old_time = time

      if (cval == 900):
        stop = timeit.default_timer()
        print('Throughput (FPS) without optimization is: ', (900 / (stop - start))) 

      q2stop = timeit.default_timer()
      q2_times.append(q2stop - q2start)

    framestop = timeit.default_timer()
    full_frame_times.append(framestop - framestart)

    return hasFrames, lis, start, old_time

#YOLO FUNCTIONALITY
def YOLO(path, cval, net, class_names, colors):
  yolostart = timeit.default_timer()

  #WEIGHTS AND DATA ARE NEEDED 
  weights = "/content/yolov3-tiny.weights"
  data = "/content/darknet/cfg/coco.data"

  #SET THE THRESHOLDS 
  nms_thresh = 0.2
  thresh = 0.2
  hier_thresh = 0.5

  #RUN THE DARKNET TINY YOLO DETECTION ON THE IMAGE 
  det = dn.detect_image(net, class_names, path.encode('utf-8'), thresh=thresh, hier_thresh=hier_thresh, nms=nms_thresh)

  #RETUNR THE IMAGE AND THE THE BB LIST OF ALL THE POSSIBLE VEHICLES
  newimage, bblist = dn.draw_boxes(det, path, colors)

  yolostop = timeit.default_timer()

  yolo_times.append(yolostop - yolostart)

  return newimage, det

#CROP THE REGION OF INTEREST FROM THE IMAGE 
def cropRegions(image, left,right,top,bottom, val):
  cropstart = timeit.default_timer()
  #SOMETIMES YOLO CAN GIVE NEGATIVE LOCATIONS WHICH WE SET TO 0 
  if left < 0:
    left = 0
  if right < 0:
    right = 0
  if top < 0:
    top = 0
  if bottom < 0:
    bottom = 0
  
  #CROP
  image = image[top:bottom, left:right]
  crop_stop = timeit.default_timer()
  crop_times.append(crop_stop - cropstart)


  return image 


def classify(img, i):
  classifierstart = timeit.default_timer()

  #WE NEED TO RESIZE THE IMAGE 
  dim = (416, 416)
  resized = cv.resize(img, dim, interpolation = cv.INTER_AREA)

  #WE NEED TO GIVE IT ANOTHER DIMENSION TO MATCH THE CLASSIFIER
  img = np.expand_dims(resized, axis=0)
  #USE OUR MODEL TO REUTN A PROBABILITY OF IT BEING A SUV AND A PROBABILITY OF IT BEING A SEDAN 
  predictions_single = probability_model.predict(img)
  #WE TAKE THE BIGGEST PROB AS OUR PREDICTION
  class_res = np.argmax(predictions_single)
  sedan = 0
  suv = 0

  #SET OUT PREDICTION VALUES THAT WE RETURN FOR OUR RESULTS
  if class_res == 0:
    sedan = 1

  elif class_res == 1:
    suv = 1

  classifierstop = timeit.default_timer()

  classifier_times.append(classifierstop - classifierstart)

  return sedan, suv

#VIDEO LOCATION
vidcap = cv.VideoCapture('/content/assignment-clip.mp4')
cval = 1

count = 1
bb = []
sec = 0

#THIS FRAME RATE GIVES US 900 IMAGES IN 30 SECONDS OF VIDEO 
frameRate = 1/30 
count=1
model = "/content/darknet/cfg/yolov3-tiny.cfg"
weights = "/content/yolov3-tiny.weights"
data = "/content/darknet/cfg/coco.data"

#WE NEED THIS NETWORK 
net, class_names, colors = dn.load_network(model, data, weights, 0)

#READ IN THE GROUND TRUTH DATA SO WE CAN COMPARE 
df = pd.read_excel("/content/gdrive/MyDrive/Groundtruth.xlsx", sheet_name = 'Sheet2')
#PUT THE RESULTS INTO THEIR OWN LISTS
frames = pd.DataFrame(df['Frame#'])
sedans = pd.DataFrame(df['Sedan'])
suvs = pd.DataFrame(df['SUV'])
total = pd.DataFrame(df['Total'])

#CONVERT TO NUMPY
total = total.to_numpy()
sedans = sedans.to_numpy()
suvs = suvs.to_numpy()
frames = frames.to_numpy()

result = []
old_time = 0
#START THE TIMER AND SET OFF THE PROCESS OF READING IN THE
start = timeit.default_timer()
success, res, start, old_time = getFrame(sec,cval, net, class_names, colors, result, start, old_time)

#LOOP UNTIL THERE ARE NO MORE FRAMES 
while success:
    cval = cval + 1
    count = count + 1
    sec = sec + frameRate
    success, res, start, old_time = getFrame(sec,cval, net, class_names, colors, result, start, old_time)




Throughput (FPS) without optimization is:  4.6511023557157785


In [68]:
print(q1_times)
print(q2_times)
print(throughput_times)
print(yolo_times)
print(classifier_times)

#WE COULD GET THE AVEAGE OF ALL THESE AND PUT THEM INTO A TABLE 

print("The average Q1 time is ", (sum(q1_times)/len(q1_times)))
print("The average Q2 time is ", (sum(q2_times)/len(q2_times)))
print("The average Throughput time is ", (sum(throughput_times)/len(throughput_times)))
print("The average YOLO time is ", (sum(yolo_times)/len(yolo_times)))
print("The average Classifier time is ", (sum(classifier_times)/len(classifier_times)))
print("The average cropping time is ", (sum(crop_times)/len(crop_times)))
#THEN LETS WRITE Q1 Q2 AND THROUGHPUT TO A .CSV FILE AND PLOT


with open('/content/results.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['Q1 Times','Q2 Times', 'Throughput'])
    for i in range(len(throughput_times)):
        resnew = ((q1_times[i]) ,q2_times[i], throughput_times[i])
        csv_out.writerow(resnew)



[0.01830045599490404, 0.01324162400123896, 0.012442182000086177, 0.013803066998661961, 0.014165005995891988, 0.013740015994699206, 0.014395976002560928, 0.01450832100090338, 0.015251813994836994, 0.016528168998775072, 0.01677397499588551, 0.016942333997576497, 0.016584436998527963, 0.04492505599773722, 0.019623940002929885, 0.020814987998164725, 0.020351218998257536, 0.020619558003090788, 0.022784559994761366, 0.021835086001374293, 0.0222445899999002, 0.023003521004284266, 0.023899684005300514, 0.02347621799708577, 0.025067940005101264, 0.02441620599711314, 0.026417772998684086, 0.02791661099763587, 0.027892811995116062, 0.04834463899896946, 0.02712104100646684, 0.028873385002953, 0.027284171999781393, 0.029901196998253, 0.030596724995120894, 0.03313338400039356, 0.03136430199811002, 0.036710789005155675, 0.03210114700050326, 0.032068118001916446, 0.03402253300009761, 0.0346495500052697, 0.03347392599971499, 0.06546436499775155, 0.036256146006053314, 0.036266991002776194, 0.04605087199

In [ ]:
#THIS IS THE FUNCTIONS FOR THE PART WITH OPTIMIZATION WITH A FEW MINOR ALTERATIONS
#NO NEED TO COMMENT AGAIN AS THERE ARE TINY DIFFEENCES 

def getFrame2(sec,cval, vid):
    done = False
    vid.set(cv.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vid.read()
    if hasFrames and cval <= 900:
      cv.imwrite("/content/gdrive/MyDrive/images/image"+str(cval)+".jpg", image)     # save frame as JPG file
      path = "/content/gdrive/MyDrive/images/image"+str(cval)+".jpg"
    else:
      path = ""
      hasFrames = False

    if (cval == 900):
      done = True
    return hasFrames, path, done


def YOLO2(path, cval, net, class_names, colors):
  weights = "/content/yolov3-tiny.weights"
  data = "/content/darknet/cfg/coco.data"

  
  nms_thresh = 0.2
  thresh = 0.2
  hier_thresh = 0.5
  
  det = dn.detect_image(net, class_names, path.encode('utf-8'), thresh=thresh, hier_thresh=hier_thresh, nms=nms_thresh)
  newimage, bblist = dn.draw_boxes(det, path, colors)
  return newimage, det

def cropRegions(image, left,right,top,bottom, val):
  if left < 0:
    left = 0
  if right < 0:
    right = 0
  if top < 0:
    top = 0
  if bottom < 0:
    bottom = 0
  image = image[top:bottom, left:right]

  return image 

def classify(img, i):

  dim = (416, 416)
  resized = cv.resize(img, dim, interpolation = cv.INTER_AREA)

  img = np.expand_dims(resized, axis=0)
  predictions_single = probability_model.predict(img)
  class_res = np.argmax(predictions_single)
  sedan = 0
  suv = 0

  if class_res == 0:
    sedan = 1

  elif class_res == 1:
    suv = 1

  return sedan, suv



This is the example with the optimization of multithreading included. 

In [71]:
import cv2 as cv
from PIL import Image
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from threading import Thread, Lock
import time
import random
import queue
import threading
from threading import Condition
import sys
import timeit
from concurrent.futures import ThreadPoolExecutor

#HERE WE USE A QUEUE WHICH MAKES IT THREAD SAFE AND ALSO SENDS NOTIFICATIONS 
#WHEN DATA HAS BEEN ADDED OR REMOVED 
q = queue.Queue()
#LIST FOR THE THREADS 
threads = list()

#ALL THE SAME AS IN LAST EXAMPLE
count = 1
bb = []

frameRate = 1/30 
count=1
model = "/content/darknet/cfg/yolov3-tiny.cfg"
weights = "/content/yolov3-tiny.weights"
data = "/content/darknet/cfg/coco.data"
net, class_names, colors = dn.load_network(model, data, weights, 0)


df = pd.read_excel("/content/gdrive/MyDrive/Groundtruth.xlsx", sheet_name = 'Sheet2')
frames = pd.DataFrame(df['Frame#'])
sedans = pd.DataFrame(df['Sedan'])
suvs = pd.DataFrame(df['SUV'])
total = pd.DataFrame(df['Total'])


total = total.to_numpy()
sedans = sedans.to_numpy()
suvs = suvs.to_numpy()
frames = frames.to_numpy()

result = []

throughput_times = []


#THIS IS THE PRODUCER THREAD WHICH IS THE VIDEO READER
#IT TAKES THE VIDEO AND PUTS THE OUTPUT WHICH IS A FRAME INTO THE QUEUE
class ProducerThread(Thread):
    Thread._is_running = True
    def run(self):
        start = timeit.default_timer()
        frameRate = 1/30 
        global queue
        sec = 0
        cval = 1
        hasFrames = True
        vid = cv.VideoCapture('/content/assignment-clip.mp4')

        #LOOP HERE WHILE THE THREAD IS ALIVE 
        while (self._is_running):

            hasFrame, path, done = getFrame2(sec,cval, vid)
            #IF WE ARE DONE END THREAD
            if (done):
              stop = timeit.default_timer()
              print('Throughput (FPS) with optimization of part 1 is: ', (900 / (stop - start))) 
              self.stop()

            sec = sec + frameRate
            cval = cval + 1
            q.put(path)

    def stop(self):
      self._is_running = False


#THIS THREAD TAKES THE FRAMES PATH OFF THE QUEUE AND RUNS YOLO, CROPS THE IMAGES
#AND RUNS THE CLASSIFIER ON EACH PROPOSED VEHICLE
class ConsumerThread(Thread):
    Thread._is_running = True
    net, class_names, colors = dn.load_network(model, data, weights, 0)

    def run(self):
        main_start = timeit.default_timer()

        start = timeit.default_timer()
        stop = ""
        global queue
        counter = 0
        output = []
        img_array = []

        while (self._is_running):
            main_start = timeit.default_timer()

            #TAKE PATH FROM QUEUE 
            path = q.get()
            if (path != ""):
              counter = counter + 1

            #RUN YOLO ON IMAGE 
            x, bb = YOLO2(path, counter, net, class_names, colors)
            image_list = []
            val = 0
            total_vehicles = 0
            sedan = 0
            suv = 0
            i = 0

            #we need to look up the image here so we can add the boxes and the labels
            frame = cv.imread("/content/gdrive/MyDrive/images/image"+str(counter)+".jpg")     # save frame as JPG file

            #LOOP THROUGH BOUNDING BOXES 
            for b in bb:
              if (b[0] == "car"):
                left, top, right, bottom = dn.bbox2points(b[2])
                image = cropRegions(x, left,right,top,bottom, val)
                val = val + 1
                total_vehicles = total_vehicles + 1
                sedanres, suvres = classify(image, i)
                label = ""
                col = (0,255,0)

                #HERE WE ANNOTATE EACH FRAME WITH THE BOUNDING BOX AND THE LABEL OF THE PREDICTION
                if (sedanres == 1):
                  label = "Sedan"
                  cv.rectangle(frame, (left, top), (right, bottom), (0,255,0), 1)
                  cv.putText(frame, "{}".format(label) , (left, top - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

                else:
                  label = "SUV"
                  cv.rectangle(frame, (left, top), (right, bottom), (0,0,128), 1)
                  cv.putText(frame, "{}".format(label) , (left, top - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,128), 2)
                

                sedan = sedan + sedanres
                suv = suv + suvres
                i = i + 1

            #GET THE RESULTS OF THE FRAME AND PUT THEM INTO A TUPLE 
            x = (sedan, suv, total_vehicles)
            output.append(x)
            img_array.append(frame)
            main_end = timeit.default_timer()

            throughput_times.append(main_end - main_start)


            #IF WE ARE DONE 
            if (counter == 900):
              stop = timeit.default_timer()
              print('Throughput (FPS) with optimization of part 2 is: ', (900 / (stop - start))) 
              throughput_times.append()
              height, width, layers = frame.shape
              size = (width, height)

              #HERE IS WHERE WE CREATE THE VIDEO 
              out = cv.VideoWriter('projectoutputvideo.avi', cv.VideoWriter_fourcc(*'DIVX'), 30, size)
              for i in range(len(img_array)):
                  out.write(img_array[i])

              out.release()
              cv.destroyAllWindows()

              self.stop()

    def stop(self):
      self._is_running = False
 
#CREATE AND START THE PRODUCER AND CONSUMER THREADS 
produce = ProducerThread()
consume = ConsumerThread()
threads.append(produce)
threads.append(consume)
produce.start()
consume.start()

for t in threads:
    t.join()


Throughput (FPS) with optimization of part 1 is:  19.659058988400083
Throughput (FPS) with optimization of part 2 is:  5.169814323080091


Exception in thread Thread-61:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-71-1640c72f5dad>", line 158, in run
    throughput_times.append()
TypeError: append() takes exactly one argument (0 given)



In [77]:
print("The average Throughput time is ", (sum(throughput_times)/len(throughput_times)))

with open('/content/results.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['Q1 Times','Q2 Times', 'Throughput'])
    for i in range(len(throughput_times)):
        resnew = ((i), (throughput_times[i]))
        csv_out.writerow(resnew)

The average Throughput time is  0.19342741207565672


In [26]:
import csv

#HERE WE CALCULATE THE F1 SCORES OF ALL THE OUTPUTS
f_pos_sedan = 0
t_pos_sedan = 0
f_neg_sedan = 0


f_pos_suv = 0
t_pos_suv = 0
f_neg_suv = 0

f_pos_total = 0
t_pos_total = 0
f_neg_total = 0

#LOOP THROUGH ALL THE RESULTS OF THE 900 FRAMES 
for i in range(len(res)):

  #CALCULATE THE F1 SCORE FOR SEDANS 
  fp_val = res[i][0] - sedans[i]
  if (fp_val >= 0):
    f_pos_sedan = f_pos_sedan + abs(fp_val)

  fn_val = sedans[i] - res[i][0]
  if (fn_val >= 0):
    f_neg_sedan = f_neg_sedan + abs(fn_val)

  if (res[i][0] >= sedans[i]):
    t_pos_sedan = t_pos_sedan + sedans[i]
  else:
    t_pos_sedan = t_pos_sedan + res[i][0]


  #CALCULATE THE F1 SCORE FOR SUVS
  fp_val = res[i][1] - suvs[i]
  if (fp_val >= 0):
    f_pos_suv = f_pos_suv + abs(fp_val)

  fn_val = suvs[i] - res[i][1]
  if (fn_val >= 0):
    f_neg_suv = f_neg_suv + abs(fn_val)

  if (res[i][1] >= suvs[i]):
    t_pos_suv = t_pos_suv + sedans[i]
  else:
    t_pos_suv = t_pos_suv + res[i][1]


  #CALCULATE THE F1 SCORE FOR THE TOTAL VEHICLES (PART 1 THE YOLO OUTPUT)
  fp_val = res[i][2] - total[i]
  if (fp_val >= 0):
    f_pos_total = f_pos_total + abs(fp_val)

  fn_val = total[i] - res[i][2]
  if (fn_val >= 0):
    f_neg_total = f_neg_total + abs(fn_val)

  if (res[i][2] >= total[i]):
    t_pos_total = t_pos_total + sedans[i]
  else:
    t_pos_total = t_pos_total + res[i][2]


#HERE WE GET THE F1 SCORE 
score_sedan = ((t_pos_sedan)/(t_pos_sedan + .5*(f_pos_sedan + f_neg_sedan)))
score_suv = ((t_pos_suv)/(t_pos_suv + .5*(f_pos_suv + f_neg_suv)))
score_total = ((t_pos_total)/(t_pos_total + .5*(f_pos_total + f_neg_total)))

print("The f score for part 1 (the YOLO detection results): ", "{:.2f}".format(score_total[0] * 100), "%")
print("The f score for the Sedan results are: ", "{:.2f}".format(score_sedan[0] * 100), "%")
print("The f score for the SUV results are: ", "{:.2f}".format(score_suv[0] * 100), "%")

print(res)

#WRITE OUR RESULTS TO A CSV FILE 
with open('/content/results_per_frame.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['Frame#','Sedan','SUV', 'Total'])
    for i in range(len(res)):
        print(res[i])
        resnew = ((i+1,) + res[i])
        csv_out.writerow(resnew)


The f score for part 1 (the YOLO detection results):  86.45 %
The f score for the Sedan results are:  73.04 %
The f score for the SUV results are:  75.08 %
[(2, 1, 3), (2, 1, 3), (1, 2, 3), (1, 2, 3), (1, 2, 3), (1, 2, 3), (1, 2, 3), (1, 2, 3), (1, 2, 3), (2, 1, 3), (2, 1, 3), (1, 2, 3), (1, 2, 3), (2, 1, 3), (0, 4, 4), (1, 3, 4), (0, 3, 3), (0, 4, 4), (1, 3, 4), (1, 3, 4), (2, 2, 4), (2, 2, 4), (1, 3, 4), (0, 4, 4), (0, 4, 4), (1, 3, 4), (0, 4, 4), (2, 2, 4), (1, 3, 4), (0, 4, 4), (1, 3, 4), (0, 4, 4), (0, 4, 4), (1, 3, 4), (0, 4, 4), (1, 3, 4), (2, 2, 4), (2, 2, 4), (1, 3, 4), (1, 3, 4), (1, 3, 4), (1, 3, 4), (1, 3, 4), (2, 2, 4), (2, 2, 4), (2, 2, 4), (2, 2, 4), (1, 3, 4), (1, 3, 4), (1, 3, 4), (1, 3, 4), (2, 2, 4), (2, 2, 4), (2, 2, 4), (2, 2, 4), (1, 2, 3), (0, 3, 3), (0, 3, 3), (0, 3, 3), (0, 3, 3), (0, 3, 3), (0, 3, 3), (0, 2, 2), (0, 2, 2), (0, 2, 2), (0, 2, 2), (0, 2, 2), (0, 3, 3), (0, 1, 1), (0, 2, 2), (0, 3, 3), (0, 1, 1), (0, 2, 2), (0, 2, 2), (0, 2, 2), (0, 2, 2), (0, 2, 